<a href="https://colab.research.google.com/github/albivaltzew/urbancode_samolet/blob/main/augment_coco_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
pip install opencv-python-headless numpy pycocotools


In [ ]:
!pip install --upgrade albumentations


In [ ]:
from pycocotools.coco import COCO



COCO_ANNO_PATH = '/content/drive/MyDrive/urbancode/urbanhack-train/annotations/instances_default.json'
COCO_IMG_PATH  = '/content/drive/MyDrive/urbancode/urbanhack-train/images'

coco = COCO(COCO_ANNO_PATH)

loading annotations into memory...
Done (t=0.98s)
creating index...
index created!


In [ ]:
import cv2
import numpy as np
import os
import albumentations as A
from google.colab.patches import cv2_imshow

In [ ]:
num_augmented_images = 1

In [ ]:
class_labels = ["window", "empty", "filled"]


# *save* in coco format

In [ ]:
# Apply augmentation techniques to images and store them
augmented_images = []
updated_annotations = []
# Perform augmentation and update annotations
l = 0
for image_id in (coco.imgs.keys()):
    l+=1
    print(l)
    image_info = coco.imgs[image_id]
    annotations = coco.loadAnns(coco.getAnnIds([image_id]))
    image = cv2.imread(f'{COCO_IMG_PATH}/{image_info["file_name"]}')
    print(f'{COCO_IMG_PATH}/{image_info["file_name"]}')

    # Apply augmentation techniques (e.g., rotation, flipping, scaling)
    bboxes = []
    class_categories = []
    for i in range(len(annotations)):
          bboxes.append(annotations[i]["bbox"])
          class_categories.append(annotations[i]['category_id'])


    transform = A.Compose([
        A.HorizontalFlip(p=0.5),
        A.Rotate(limit=30, p=0.2),
        A.OneOf([
            A.RandomBrightnessContrast(p=0.5),
            A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2, p=0.5), # изменение цвета
            A.augmentations.transforms.GaussNoise(var_limit=(10,50),mean=0,p=0.5), # шум гаусса
            A.augmentations.transforms.CLAHE (clip_limit=4.0, tile_grid_size=(8, 8), always_apply=False, p=0.5)
        ], p=0.3),
        A.Blur(blur_limit=3, p=0.1), # Размытие
        # A.augmentations.transforms.Equalize(),
        A.augmentations.geometric.transforms.Affine(p=0.1, shear={'x': (-30, 30), 'y': (-30, 30)}),
        A.OneOf([
            A.augmentations.transforms.ToGray(p=0.2), # оттенки серого
            A.RandomRain(brightness_coefficient=0.9, drop_width=1, blur_value=3, p=0.2), # добавляет дождь
            A.RandomSnow(brightness_coeff=2.5, snow_point_lower=0.3, snow_point_upper=0.5, p=0.2), #Добавляет снег
            A.RandomFog(fog_coef_lower=0.3, fog_coef_upper=0.6, alpha_coef=0.1, p=0.2), # туман
            A.RandomSunFlare(flare_roi=(0, 0, 1, 0.5), angle_lower=0.5, p=0.22), # солнечные блики
        ],p=0.3),
    ], bbox_params=A.BboxParams(format='coco', min_area=1024, min_visibility=0.3, label_fields=['class_categories']))

    for _ in range(num_augmented_images):
        augmented = transform(image=image, bboxes=bboxes,  class_categories=class_categories)
        augmented_image = augmented['image']
        augmented_bboxes = augmented['bboxes']
        augmented_categories = augmented['class_categories']
        augmented_images.append(augmented_image)


        # Create a new annotation entry for the augmented image
        updated_annotation = []
        for i in range(len(augmented_bboxes)):

          updated_ann = {
              'image_id': annotations[i]['image_id'],
              'category_id': augmented_categories[i],
              'id':i+1,  # Make sure to keep the original ID
              'bbox': augmented_bboxes[i],
              'area': augmented_bboxes[i][2]*augmented_bboxes[i][3],  # You may need to adjust this depending on the augmentation
              'segmentation': annotations[i]['segmentation'],  # You may need to adjust this depending on the augmentation
              'iscrowd': annotations[i]['iscrowd'],  # You may need to adjust this depending on the augmentation
          }

          # Append updated annotation to the list
          updated_annotation.append(updated_ann)
    updated_annotations.append(updated_annotation)




In [ ]:
import json

In [ ]:
# Create updated annotations dictionary
updated_annotations_data = {
    'images': coco.dataset['images'],
    'annotations': updated_annotations,
    'categories': coco.dataset['categories']
}

# Save the updated annotations in COCO format
with open('/content/drive/MyDrive/urbancode/urbanhack-train/updated_annotations.json', 'w') as f:
    json.dump(updated_annotations_data, f)


In [ ]:
augmented_images_directory = "/content/drive/MyDrive/urbancode/urbanhack-train/augmented_images_new"
os.mkdir(augmented_images_directory)
for i, augmented_image in enumerate(augmented_images):
    cv2.imwrite(os.path.join(augmented_images_directory, f"augmented_image_{i}.jpg"), augmented_image)


# *save* in yolo

In [ ]:
import os

# Define the function to convert COCO annotations to YOLO format
def coco_to_yolo(bbox, category_id, image_width, image_height):
    x_center = (bbox[0] + bbox[2]) / 2
    y_center = (bbox[1] + bbox[3]) / 2
    width = bbox[2]
    height = bbox[3]

    # Normalize coordinates
    x_center /= image_width
    y_center /= image_height
    width /= image_width
    height /= image_height

    return f"{category_id} {x_center} {y_center} {width} {height}"



In [ ]:
# Create a directory to store YOLO annotations
augmented_labels_directory  = "/content/drive/MyDrive/urbancode/urbanhack-train/augmented_data_yolo/labels"
os.makedirs(augmented_labels_directory , exist_ok=True)
augmented_images_directory = "/content/drive/MyDrive/urbancode/urbanhack-train/augmented_data_yolo/images"
os.mkdir(augmented_images_directory)

In [ ]:
# Apply augmentation techniques to images and store them
augmented_images = []
# Perform augmentation and update annotations
for (l, image_id) in enumerate(coco.imgs.keys()):
    image_info = coco.imgs[image_id]
    annotations = coco.loadAnns(coco.getAnnIds([image_id]))
    image = cv2.imread(f'{COCO_IMG_PATH}/{image_info["file_name"]}')

    # Apply augmentation techniques (e.g., rotation, flipping, scaling)
    bboxes = []
    class_categories = []
    for i in range(len(annotations)):
          bboxes.append(annotations[i]["bbox"])
          class_categories.append(annotations[i]['category_id'])


    transform = A.Compose([
        A.HorizontalFlip(p=0.5),
        A.Rotate(limit=30, p=0.2),
        A.OneOf([
            A.RandomBrightnessContrast(p=0.5),
            A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2, p=0.5), # изменение цвета
            A.augmentations.transforms.GaussNoise(var_limit=(10,50),mean=0,p=0.5), # шум гаусса
            A.augmentations.transforms.CLAHE (clip_limit=4.0, tile_grid_size=(8, 8), always_apply=False, p=0.5)
        ], p=0.3),
        A.Blur(blur_limit=3, p=0.1), # Размытие
        # A.augmentations.transforms.Equalize(),
        A.augmentations.geometric.transforms.Affine(p=0.1, shear={'x': (-30, 30), 'y': (-30, 30)}),
        A.OneOf([
            A.augmentations.transforms.ToGray(p=0.2), # оттенки серого
            A.RandomRain(brightness_coefficient=0.9, drop_width=1, blur_value=3, p=0.2), # добавляет дождь
            A.RandomSnow(brightness_coeff=2.5, snow_point_lower=0.3, snow_point_upper=0.5, p=0.2), #Добавляет снег
            A.RandomFog(fog_coef_lower=0.3, fog_coef_upper=0.6, alpha_coef=0.1, p=0.2), # туман
            A.RandomSunFlare(flare_roi=(0, 0, 1, 0.5), angle_lower=0.5, p=0.22), # солнечные блики
        ],p=0.3),
    ], bbox_params=A.BboxParams(format='coco', min_area=1024, min_visibility=0.3, label_fields=['class_categories']))

    for j in range(num_augmented_images):
        augmented = transform(image=image, bboxes=bboxes,  class_categories=class_categories)
        augmented_image = augmented['image']
        augmented_bboxes = augmented['bboxes']
        augmented_categories = augmented['class_categories']
        augmented_images.append(augmented_image)

        # Iterate over annotations and save YOLO format annotations
        for i in range(len(augmented_bboxes)):

            image_width = augmented_image.shape[0]
            image_height = augmented_image.shape[1]
            bbox = augmented_bboxes[i]
            category_id = augmented_categories[i]
            yolo_annotation = coco_to_yolo(bbox, category_id, image_width, image_height)


            # Create an annotation file for each image
            num_im = l+j
            annotation_file_name = os.path.join(augmented_labels_directory, (f"augmented_image_{num_im}" + ".txt"))
            # Write YOLO annotation to file
            with open(annotation_file_name, 'a') as f:
                f.write(yolo_annotation + '\n')

            cv2.imwrite(os.path.join(augmented_images_directory, f"augmented_image_{num_im}.jpg"), augmented_image)





KeyboardInterrupt: ignored